In [1]:
import catboost as cb
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import xgboost as xgb

train = pd.read_csv("../titanic/data/train.csv")
test = pd.read_csv("../titanic/data/test.csv")

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
def dataprep(data):
    cabin = data['Cabin'].fillna('X').str[0]
    # Age 
    age = data['Age'].fillna(data['Age'].mean())
    emb = data['Embarked'].fillna('X').str[0]
    # Prix du billet / Attention une donnée de test n'a pas de Prix !
    faresc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Fare']].fillna(0)), columns = ['Prix'])
    # Classe
    pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])
    
    dp = data[['SibSp', 'Sex']].join(pc).join(faresc).join(age).join(cabin).join(emb)
    return dp

In [4]:
Xtrain = dataprep(train)
Xtest = dataprep(test)
y = train.Survived

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
Xtrain.head()

,SibSp,Sex,Classe,Prix,Age,Cabin,Embarked
0,1,male,1.0,0.014151,22.0,X,S
1,1,female,0.0,0.139136,38.0,C,C
2,0,female,1.0,0.015469,26.0,X,S
3,1,female,0.0,0.103644,35.0,C,S
4,0,male,1.0,0.015713,35.0,X,S


In [6]:
Xtrain.describe()

,SibSp,Classe,Prix,Age
count,891.000000,891.000000,891.000000,891.000000
mean,0.523008,0.654321,0.062858,29.699118
std,1.102743,0.418036,0.096995,13.002015
min,0.000000,0.000000,0.000000,0.420000
25%,0.000000,0.500000,0.015440,22.000000
50%,0.000000,1.000000,0.028213,29.699118
75%,1.000000,1.000000,0.060508,35.000000
max,8.000000,1.000000,1.000000,80.000000


In [7]:
cat_features = [1,5,6]
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
          'l2_leaf_reg': [1,4,9],
          'iterations': [300]}
cbc = cb.CatBoostClassifier()
cbc.fit(Xtrain, y, cat_features)

Learning rate set to 0.016216
0:	learn: 0.6840516	total: 57.7ms	remaining: 57.6s
1:	learn: 0.6753065	total: 61ms	remaining: 30.4s
2:	learn: 0.6665325	total: 64.7ms	remaining: 21.5s
3:	learn: 0.6579853	total: 70.1ms	remaining: 17.5s
4:	learn: 0.6495075	total: 75.1ms	remaining: 14.9s
5:	learn: 0.6444749	total: 77.8ms	remaining: 12.9s
6:	learn: 0.6396174	total: 80ms	remaining: 11.3s
7:	learn: 0.6302390	total: 85.1ms	remaining: 10.5s
8:	learn: 0.6206268	total: 90.1ms	remaining: 9.93s
9:	learn: 0.6141353	total: 93.6ms	remaining: 9.27s
10:	learn: 0.6082755	total: 98.7ms	remaining: 8.87s
11:	learn: 0.6043012	total: 101ms	remaining: 8.32s
12:	learn: 0.5973774	total: 106ms	remaining: 8.05s
13:	learn: 0.5904514	total: 112ms	remaining: 7.85s
14:	learn: 0.5850055	total: 115ms	remaining: 7.54s
15:	learn: 0.5793372	total: 118ms	remaining: 7.27s
16:	learn: 0.5738179	total: 124ms	remaining: 7.14s
17:	learn: 0.5673118	total: 129ms	remaining: 7.01s
18:	learn: 0.5604090	total: 134ms	remaining: 6.9s
19:	l

192:	learn: 0.3950213	total: 973ms	remaining: 4.07s
193:	learn: 0.3949342	total: 977ms	remaining: 4.06s
194:	learn: 0.3946580	total: 985ms	remaining: 4.07s
195:	learn: 0.3938872	total: 990ms	remaining: 4.06s
196:	learn: 0.3938389	total: 994ms	remaining: 4.05s
197:	learn: 0.3936847	total: 998ms	remaining: 4.04s
198:	learn: 0.3935977	total: 1s	remaining: 4.03s
199:	learn: 0.3934991	total: 1.01s	remaining: 4.04s
200:	learn: 0.3932966	total: 1.01s	remaining: 4.03s
201:	learn: 0.3927915	total: 1.02s	remaining: 4.03s
202:	learn: 0.3927195	total: 1.02s	remaining: 4.01s
203:	learn: 0.3925769	total: 1.03s	remaining: 4.01s
204:	learn: 0.3924996	total: 1.03s	remaining: 4s
205:	learn: 0.3921331	total: 1.03s	remaining: 3.99s
206:	learn: 0.3917746	total: 1.04s	remaining: 3.98s
207:	learn: 0.3916702	total: 1.04s	remaining: 3.97s
208:	learn: 0.3915266	total: 1.04s	remaining: 3.96s
209:	learn: 0.3912927	total: 1.05s	remaining: 3.96s
210:	learn: 0.3912640	total: 1.06s	remaining: 3.95s
211:	learn: 0.3911

397:	learn: 0.3729665	total: 1.76s	remaining: 2.66s
398:	learn: 0.3729577	total: 1.76s	remaining: 2.65s
399:	learn: 0.3728200	total: 1.77s	remaining: 2.65s
400:	learn: 0.3725122	total: 1.78s	remaining: 2.65s
401:	learn: 0.3725013	total: 1.78s	remaining: 2.65s
402:	learn: 0.3724873	total: 1.78s	remaining: 2.64s
403:	learn: 0.3724386	total: 1.79s	remaining: 2.64s
404:	learn: 0.3723384	total: 1.79s	remaining: 2.63s
405:	learn: 0.3722989	total: 1.8s	remaining: 2.63s
406:	learn: 0.3722944	total: 1.8s	remaining: 2.62s
407:	learn: 0.3722252	total: 1.81s	remaining: 2.62s
408:	learn: 0.3721943	total: 1.81s	remaining: 2.62s
409:	learn: 0.3721943	total: 1.81s	remaining: 2.61s
410:	learn: 0.3721777	total: 1.82s	remaining: 2.6s
411:	learn: 0.3721601	total: 1.82s	remaining: 2.6s
412:	learn: 0.3720607	total: 1.82s	remaining: 2.59s
413:	learn: 0.3720008	total: 1.83s	remaining: 2.59s
414:	learn: 0.3717595	total: 1.83s	remaining: 2.58s
415:	learn: 0.3715670	total: 1.84s	remaining: 2.58s
416:	learn: 0.37

571:	learn: 0.3540042	total: 2.54s	remaining: 1.9s
572:	learn: 0.3539870	total: 2.54s	remaining: 1.9s
573:	learn: 0.3538132	total: 2.55s	remaining: 1.89s
574:	learn: 0.3535254	total: 2.56s	remaining: 1.89s
575:	learn: 0.3533917	total: 2.56s	remaining: 1.89s
576:	learn: 0.3533760	total: 2.56s	remaining: 1.88s
577:	learn: 0.3530526	total: 2.57s	remaining: 1.88s
578:	learn: 0.3529776	total: 2.58s	remaining: 1.87s
579:	learn: 0.3529661	total: 2.58s	remaining: 1.87s
580:	learn: 0.3528552	total: 2.59s	remaining: 1.86s
581:	learn: 0.3527708	total: 2.59s	remaining: 1.86s
582:	learn: 0.3527239	total: 2.6s	remaining: 1.86s
583:	learn: 0.3526241	total: 2.6s	remaining: 1.85s
584:	learn: 0.3525667	total: 2.6s	remaining: 1.85s
585:	learn: 0.3524633	total: 2.61s	remaining: 1.84s
586:	learn: 0.3522500	total: 2.61s	remaining: 1.84s
587:	learn: 0.3521454	total: 2.62s	remaining: 1.83s
588:	learn: 0.3520261	total: 2.62s	remaining: 1.83s
589:	learn: 0.3518920	total: 2.63s	remaining: 1.83s
590:	learn: 0.351

754:	learn: 0.3333076	total: 3.51s	remaining: 1.14s
755:	learn: 0.3332608	total: 3.52s	remaining: 1.14s
756:	learn: 0.3332054	total: 3.52s	remaining: 1.13s
757:	learn: 0.3332023	total: 3.53s	remaining: 1.13s
758:	learn: 0.3329741	total: 3.54s	remaining: 1.12s
759:	learn: 0.3328579	total: 3.54s	remaining: 1.12s
760:	learn: 0.3327801	total: 3.55s	remaining: 1.11s
761:	learn: 0.3326261	total: 3.55s	remaining: 1.11s
762:	learn: 0.3325717	total: 3.56s	remaining: 1.1s
763:	learn: 0.3324600	total: 3.56s	remaining: 1.1s
764:	learn: 0.3323292	total: 3.57s	remaining: 1.09s
765:	learn: 0.3322550	total: 3.57s	remaining: 1.09s
766:	learn: 0.3322052	total: 3.58s	remaining: 1.09s
767:	learn: 0.3321525	total: 3.58s	remaining: 1.08s
768:	learn: 0.3320781	total: 3.59s	remaining: 1.08s
769:	learn: 0.3320390	total: 3.59s	remaining: 1.07s
770:	learn: 0.3319387	total: 3.6s	remaining: 1.07s
771:	learn: 0.3318848	total: 3.6s	remaining: 1.06s
772:	learn: 0.3318471	total: 3.61s	remaining: 1.06s
773:	learn: 0.33

917:	learn: 0.3187720	total: 4.48s	remaining: 400ms
918:	learn: 0.3186112	total: 4.49s	remaining: 396ms
919:	learn: 0.3182064	total: 4.5s	remaining: 391ms
920:	learn: 0.3179346	total: 4.51s	remaining: 387ms
921:	learn: 0.3177056	total: 4.51s	remaining: 382ms
922:	learn: 0.3176681	total: 4.52s	remaining: 377ms
923:	learn: 0.3176404	total: 4.53s	remaining: 372ms
924:	learn: 0.3176325	total: 4.53s	remaining: 368ms
925:	learn: 0.3175225	total: 4.54s	remaining: 363ms
926:	learn: 0.3174864	total: 4.55s	remaining: 359ms
927:	learn: 0.3173563	total: 4.56s	remaining: 354ms
928:	learn: 0.3171236	total: 4.57s	remaining: 349ms
929:	learn: 0.3169208	total: 4.58s	remaining: 344ms
930:	learn: 0.3169171	total: 4.58s	remaining: 339ms
931:	learn: 0.3169065	total: 4.58s	remaining: 335ms
932:	learn: 0.3168353	total: 4.59s	remaining: 330ms
933:	learn: 0.3168248	total: 4.6s	remaining: 325ms
934:	learn: 0.3168210	total: 4.6s	remaining: 320ms
935:	learn: 0.3167818	total: 4.61s	remaining: 315ms
936:	learn: 0.3

In [8]:
p_cbc = cbc.predict(Xtrain)
print ("Score Train -->", round(cbc.score(Xtrain, y) *100,2), " %")

Score Train --> 87.77  %


In [9]:
clf = cb.CatBoostClassifier(eval_metric="AUC", 
                            depth=10, 
                            iterations= 500, 
                            l2_leaf_reg= 9, 
                            learning_rate= 0.15)
clf.fit(Xtrain, y, cat_features)

0:	total: 3.28ms	remaining: 1.64s
1:	total: 8.06ms	remaining: 2.01s
2:	total: 20.9ms	remaining: 3.45s
3:	total: 24.6ms	remaining: 3.05s
4:	total: 29.2ms	remaining: 2.89s
5:	total: 32.2ms	remaining: 2.65s
6:	total: 35.1ms	remaining: 2.47s
7:	total: 104ms	remaining: 6.38s
8:	total: 110ms	remaining: 6.02s
9:	total: 117ms	remaining: 5.73s
10:	total: 121ms	remaining: 5.38s
11:	total: 125ms	remaining: 5.08s
12:	total: 132ms	remaining: 4.94s
13:	total: 136ms	remaining: 4.71s
14:	total: 139ms	remaining: 4.51s
15:	total: 143ms	remaining: 4.34s
16:	total: 178ms	remaining: 5.05s
17:	total: 180ms	remaining: 4.83s
18:	total: 184ms	remaining: 4.66s
19:	total: 189ms	remaining: 4.53s
20:	total: 192ms	remaining: 4.38s
21:	total: 195ms	remaining: 4.24s
22:	total: 199ms	remaining: 4.13s
23:	total: 211ms	remaining: 4.18s
24:	total: 214ms	remaining: 4.07s
25:	total: 219ms	remaining: 3.99s
26:	total: 224ms	remaining: 3.93s
27:	total: 231ms	remaining: 3.9s
28:	total: 236ms	remaining: 3.84s
29:	total: 242ms	r

242:	total: 11.4s	remaining: 12s
243:	total: 11.4s	remaining: 12s
244:	total: 11.5s	remaining: 12s
245:	total: 11.6s	remaining: 11.9s
246:	total: 11.6s	remaining: 11.9s
247:	total: 11.7s	remaining: 11.8s
248:	total: 11.7s	remaining: 11.8s
249:	total: 11.8s	remaining: 11.8s
250:	total: 11.9s	remaining: 11.8s
251:	total: 12s	remaining: 11.8s
252:	total: 12s	remaining: 11.7s
253:	total: 12.1s	remaining: 11.7s
254:	total: 12.2s	remaining: 11.7s
255:	total: 12.3s	remaining: 11.7s
256:	total: 12.3s	remaining: 11.7s
257:	total: 12.4s	remaining: 11.7s
258:	total: 12.5s	remaining: 11.6s
259:	total: 12.5s	remaining: 11.5s
260:	total: 12.6s	remaining: 11.5s
261:	total: 12.6s	remaining: 11.4s
262:	total: 12.6s	remaining: 11.4s
263:	total: 12.6s	remaining: 11.3s
264:	total: 12.7s	remaining: 11.3s
265:	total: 12.8s	remaining: 11.2s
266:	total: 12.8s	remaining: 11.2s
267:	total: 12.9s	remaining: 11.1s
268:	total: 12.9s	remaining: 11.1s
269:	total: 13s	remaining: 11.1s
270:	total: 13.1s	remaining: 11.

482:	total: 26.8s	remaining: 943ms
483:	total: 26.9s	remaining: 888ms
484:	total: 26.9s	remaining: 831ms
485:	total: 26.9s	remaining: 776ms
486:	total: 26.9s	remaining: 719ms
487:	total: 27s	remaining: 664ms
488:	total: 27.1s	remaining: 609ms
489:	total: 27.1s	remaining: 554ms
490:	total: 27.2s	remaining: 499ms
491:	total: 27.3s	remaining: 444ms
492:	total: 27.3s	remaining: 388ms
493:	total: 27.4s	remaining: 333ms
494:	total: 27.5s	remaining: 278ms
495:	total: 27.6s	remaining: 222ms
496:	total: 27.6s	remaining: 167ms
497:	total: 27.7s	remaining: 111ms
498:	total: 27.7s	remaining: 55.5ms
499:	total: 27.8s	remaining: 0us


In [10]:
print ("Score Train -->", round(clf.score(Xtrain, y) *100,2), " %")

Score Train --> 92.59  %


In [19]:
cb.CatBoost.save_model(clf, 
                       "catboost.modele", 
                       format="cbm", 
                       export_parameters=None, 
                       pool=None)

In [28]:
from catboost import CatBoostClassifier
clf2 = CatBoostClassifier()
clf2.load_model(fname="catboost.modele", format="cbm")

In [29]:
print ("Score Train -->", round(clf2.score(Xtrain, y) *100,2), " %")

Score Train --> 92.59  %
